# PC4 (ENTREGABLE 1)

## **Proyecto 5: Desplegar una API de preguntas y respuestas basada en LLM finamente ajustado**

## **Proyecto 5: Desplegar una API de preguntas y respuestas basada en LLM finamente ajustado**

### **Entregable 1 - Fecha: 30 de noviembre**

**Trabajo (8 puntos)**

| Criterio                                                                | Puntos |
|-------------------------------------------------------------------------|--------|
| Ajuste fino del LLM para preguntas y respuestas                         | 3      |
| Indexación y organización eficiente de la base de datos de documentos   | 3      |
| Implementación inicial de búsqueda eficiente en los documentos          | 2      |
| **Total**                                                               | **8**  |

**Exposición (12 puntos)**

| Criterio                                                                 | Puntos |
|--------------------------------------------------------------------------|--------|
| Descripción del LLM y su ajuste para la tarea específica                 | 4      |
| Explicación de la base de datos y mecanismos de búsqueda implementados   | 4      |
| Claridad en la presentación y capacidad para responder preguntas         | 4      |
| **Total**                                                                | **12** |

In [ ]:
!nvidia-smi

Sat Dec  7 02:05:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              46W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
pip install transformers fastapi uvicorn elasticsearch sentence-transformers datasets

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, Trainer, TrainingArguments, pipeline
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW, get_linear_schedule_with_warmup
from datasets import load_dataset
import torch
from torch.utils.data import DataLoader, RandomSampler
from tqdm import tqdm
from transformers.data.processors.squad import SquadProcessor
from transformers import squad_convert_examples_to_features

In [ ]:
num_threads = torch.get_num_threads()
print(f"Current number of threads: {num_threads}")

Current number of threads: 1


In [ ]:
torch.set_num_threads(8)  # Set number of threads for PyTorch
torch.set_num_interop_threads(2)  # Optional: Set inter-op parallelism threads

In [ ]:
print(f"Current number of threads: {torch.get_num_threads()}")

Current number of threads: 8


## SQUAD

Se importa dataset SQUAD para el entrenamiento de BERT basado en preguntas y respuestas

In [ ]:
dataset = load_dataset("squad_v2")
#dataset["train"] = dataset["train"].select(range(len(dataset["train"])))
#dataset["validation"] = dataset["validation"].select(range(len(dataset["validation"])))

Estructura de los datos de entrenamiento

In [ ]:
len(dataset["train"])

130319

Se define una función para obtener la siguiente estructura:

```
{
  input_ids:
  tokenized_question and answer:
  answer_start_pos:
  answer_end_pos:
}
```


Se usará el modelo large de Bert https://huggingface-co.translate.goog/google-bert/bert-large-uncased



In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments


MODEL_NAME = "bert-large-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


def preprocess_function_with_labels(examples):
    inputs = tokenizer(
        examples["question"],
        examples["context"],
        truncation=True,  #trunca los inputs con una longitud max_length
        padding="max_length",   #modifica la longitud de los inputs para que sea igual a max_length
        max_length=384,
        stride=128,   #se segmenta inputs mayores a max_length de manera que se conserven los 'stride' tokens anteriores
        return_overflowing_tokens=True,
        return_offsets_mapping=True,    #retorna los mapeos entre posiciones de caracteres y tokens
        return_tensors="pt",    #formato pytorch
    )

    sample_mapping = inputs.pop("overflow_to_sample_mapping")
    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]

    start_positions = []
    end_positions = []

    #para cada offset mapeado entre tokens y caracteres de las respuestas
    for i, offsets in enumerate(offset_mapping):

        sample_idx = sample_mapping[i]
        answer = answers[sample_idx]

        #Si no hay respuesta
        if len(answer["answer_start"]) == 0:
            start_positions.append(0)
            end_positions.append(0)
        else:

            start_char = answer["answer_start"][0]
            end_char = start_char + len(answer["text"][0])

            #Tokens de inicio y fin de la respuesta dentro del contexto
            token_start = 0
            token_end = 0
            for idx, (start, end) in enumerate(offsets):
                if start <= start_char < end:
                    token_start = idx
                if start < end_char <= end:
                    token_end = idx
                    break

            start_positions.append(token_start)
            end_positions.append(token_end)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

#Se aplica el preprocesamiento a todo el dataset
tokenized_datasets = dataset.map(
    preprocess_function_with_labels,
    batched=True,
    remove_columns=dataset["train"].column_names,
)


Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [ ]:
from pprint import pp
pp(tokenized_datasets["train"].features)
print(len(tokenized_datasets["train"]))

{'input_ids': Sequence(feature=Value(dtype='int32', id=None),
                       length=-1,
                       id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None),
                            length=-1,
                            id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None),
                            length=-1,
                            id=None),
 'start_positions': Value(dtype='int64', id=None),
 'end_positions': Value(dtype='int64', id=None)}
65778


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
print(torch.cuda.memory_summary(device="cuda"))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 1            |        cudaMalloc retries: 1         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   9771 MiB |  14875 MiB |    951 GiB |    941 GiB |
|       from large pool |   9757 MiB |  14855 MiB |    945 GiB |    935 GiB |
|       from small pool |     13 MiB |     20 MiB |      6 GiB |      6 GiB |
|---------------------------------------------------------------------------|
| Active memory         |   9771 MiB |  14875 MiB |    951 GiB |    941 GiB |
|       from large pool |   9757 MiB |  14855 MiB |    945 GiB |

Entrenamiento del modelo con el dataset transformado

In [ ]:
import os


os.environ["WANDB_DISABLED"] = "true"


model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)
model = model.to(device)


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    weight_decay=0.01,
    save_steps=2000,
    logging_dir="./logs",
    logging_steps=1000,
    fp16=True
)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer
)


trainer.train()


model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-9-777285895f7f>:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,1.159900,1.016334
2,0.840200,1.136500
3,0.613800,1.296090
4,0.461400,1.383938
5,0.368900,1.609051


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.txt',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [ ]:
model.save_pretrained("./fine_tuned_model", safe_serialization=False)

In [ ]:
!zip -r fine_tuned_model.zip fine_tuned_model

  adding: fine_tuned_model/ (stored 0%)
  adding: fine_tuned_model/tokenizer.json (deflated 71%)
  adding: fine_tuned_model/model.safetensors (deflated 7%)
  adding: fine_tuned_model/special_tokens_map.json (deflated 42%)
  adding: fine_tuned_model/config.json (deflated 47%)
  adding: fine_tuned_model/vocab.txt (deflated 53%)
  adding: fine_tuned_model/tokenizer_config.json (deflated 76%)
  adding: fine_tuned_model/pytorch_model.bin (deflated 7%)


In [ ]:
def answer_question(question, context):

    inputs = tokenizer.encode_plus(
        question,
        context,
        add_special_tokens=True,
        return_tensors="pt"
    ).to(device)


    input_ids = inputs["input_ids"].tolist()[0]
    outputs = model(**inputs)


    answer_start = outputs.start_logits.argmax()
    answer_end = outputs.end_logits.argmax() + 1


    answer = tokenizer.decode(input_ids[answer_start:answer_end])
    return answer

In [ ]:
question = "Who created BERT?"
context = (
    "Bidirectional encoder representations from transformers BERT is a language model introduced in October 2018 by researchers at Google. It learns to represent text as a sequence of vectors using self-supervised learning. It uses the encoder-only transformer architecture. It is notable for its dramatic improvement over previous state-of-the-art models, and as an early example of a large language model"
)

# Get the answer
answer = answer_question(question, context)
print(f"Question: {question}")
print(f"Answer: {answer}")

Question: Who created BERT?
Answer: researchers at google


## Database

Base de datos de libros de ciencia e ingenieria:
https://huggingface.co/datasets/open-phi/textbooks

In [ ]:
#Dataset de textos de libros de ciencias
db = load_dataset("open-phi/textbooks")

In [ ]:
import re
def split_markdown_by_headers(markdown_text):
    """
    Divide el texto de markdown según sus encabezados
    """
    # Regex que selecciona los títulos
    header_regex = re.compile(r"^(#{2})\s+(.+)", re.MULTILINE)


    matches = list(header_regex.finditer(markdown_text))

    sections = {}
    for i, match in enumerate(matches):
        header = match.group(2).strip()
        start = match.end()              # Inicio del contenido de la sección identificada

        # Final de la sección
        end = matches[i + 1].start() if i + 1 < len(matches) else len(markdown_text)
        content = markdown_text[start:end].strip()

        sections[header] = content

    return sections

In [ ]:
#EJEMPLO
for t in db["train"]:
  print(t["topic"])

Computational Cognitive Science.
Multi-Scale System Design.
Introduction to Modeling and Simulation.
Human Supervisory Control of Automated Systems.
Numerical Computation for Mechanical Engineers.
Modeling and Simulation of Dynamic Systems.
Systems Optimization.
Modern Navigation.
Thermodynamics & Kinetics.
Electromagnetic Fields,  Forces,  and Motion.
Introductory Analog Electronics Laboratory.
Numerical Fluid Mechanics.
Urban Design Skills: Observing,  Interpreting,  and Representing the City.
Theory of Solids I.
Quasi-Balanced Circulations in Oceans and Atmospheres.
Dynamics and Vibration (13.013J).
Readings in Optimization.
Groundwater Hydrology.
Computational Cognitive Science.
Global Freshwater Crisis.
Optimization Methods.
System Dynamics II.
Atmospheric and Oceanic Modeling.
Multi-Scale System Design.
Seeing the City Afresh.
Numerical Computation for Mechanical Engineers.
Organizing for Innovative Product Development.
Mechanical Behavior of Plastics.
Information and Communicati

In [ ]:
from pprint import pp
sections = split_markdown_by_headers(db["train"][261]['markdown'])

for key, value in sections.items():
  print(key)
  print(len(value))
  # print(value)
# pp(db["train"][227 ]["outline"])

Foreward
2230
Chapter: Modeling Environmental Complexity: A Comprehensive Guide
2284
Chapter 1: Introduction:
10553
Chapter 4: Self-organized Criticality:
8591
Chapter 5: Anomalous Diffusion:
27552
Chapter 11: Environmental Complexity and Ecosystem Services:
9431
Chapter 17: Environmental Complexity and Agriculture:
8072
Chapter 18: Environmental Complexity and Energy Systems:
51005
Chapter 20: Environmental Complexity and Policy Making:
3998


In [ ]:
!pip install elasticsearch

### Elasticsearch

Código de prueba para verificar el estado de elasticsearch

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import ConnectionError

es = Elasticsearch(["https://mrkite-elasticsearch.hf.space"], timeout=30, max_retries=10, retry_on_timeout=True)

try:
    health = es.cluster.health()
    print("Elasticsearch Cluster Health:", health)

    version = es.info()
    print("Elasticsearch Version:", version['version']['number'])


    doc = {
        'author': 'John Doe',
        'text': 'Elasticsearch is fun!',
        'timestamp': '2024-11-26'
    }

    response = es.index(index="test_index", id=1, document=doc)
    print("Document indexed:", response)

    retrieved_doc = es.get(index="test_index", id=1)
    print("Retrieved Document:", retrieved_doc['_source'])

except ConnectionError:
    print("Error: Could not connect to Elasticsearch. Please make sure the server is running.")
except Exception as e:
    print("An error occurred:", e)


<ipython-input-231-ae30c01fd2f4>:4: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch(["https://mrkite-elasticsearch.hf.space"], timeout=30, max_retries=10, retry_on_timeout=True)


Elasticsearch Cluster Health: {'cluster_name': 'docker-cluster', 'status': 'green', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 0, 'active_shards': 0, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 0, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 100.0}
Elasticsearch Version: 8.15.5
Document indexed: {'_index': 'test_index', '_id': '1', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}
Retrieved Document: {'author': 'John Doe', 'text': 'Elasticsearch is fun!', 'timestamp': '2024-11-26'}


In [ ]:
from elasticsearch import Elasticsearch

es = Elasticsearch(["https://mrkite-elasticsearch.hf.space"], timeout=30, max_retries=10, retry_on_timeout=True)

index_name = "textbooks"

# Se define el mapeo de índices para el índice textbooks
mapping = {
    "mappings": {
        "properties": {
            "concepts": {
                "type": "text",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "ignore_above": 256
                    }
                }
            },
            "markdown": {
                "type": "nested",  # Define the 'markdown' field as 'nested'
                "properties": {
                    "chapter": {
                        "type": "text",
                        "fields": {
                            "keyword": {
                                "type": "keyword",
                                "ignore_above": 256
                            }
                        }
                    },
                    "content": {
                        "type": "text"
                    }
                }
            },
            "model": {
                "type": "text",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "ignore_above": 256
                    }
                }
            },
            "outline": {
                "type": "text",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "ignore_above": 256
                    }
                }
            }
        }
    }
}

# Create or update the index with the mapping
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=mapping)
    print(f"Index '{index_name}' created with nested mapping.")
else:
    print(f"Index '{index_name}' already exists.")

<ipython-input-233-00c312afb6d3>:4: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch(["https://mrkite-elasticsearch.hf.space"], timeout=30, max_retries=10, retry_on_timeout=True)


Index 'textbooks' already exists.


In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import ConnectionError

#SE INDEXAN LOS DOCUMENTOS EN ELASTICSEARCH

es = Elasticsearch(["https://mrkite-elasticsearch.hf.space"], timeout=30, max_retries=10, retry_on_timeout=True)


try:
    for id, doc in enumerate(db["train"]):
        markdown_sections = split_markdown_by_headers(doc["markdown"])
        doc["markdown"] = [{"chapter": header, "content": content} for header, content in markdown_sections.items()]
        response = es.index(index="textbooks", id=id, document=doc)
        print("Document indexed:", response)

except ConnectionError:
    print("Error: Could not connect to Elasticsearch. Please make sure the server is running.")
except Exception as e:
    print("An error occurred:", e)


<ipython-input-234-17192fd2f351>:5: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch(["https://mrkite-elasticsearch.hf.space"], timeout=30, max_retries=10, retry_on_timeout=True)


Elasticsearch Cluster Health: {'cluster_name': 'docker-cluster', 'status': 'yellow', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 2, 'active_shards': 2, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 2, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 50.0}
Elasticsearch Version: 8.15.5
Document indexed: {'_index': 'textbooks', '_id': '0', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}
Document indexed: {'_index': 'textbooks', '_id': '1', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}
Document indexed: {'_index': 'textbooks', '_id': '2', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_p

In [ ]:
response = es.count(index="textbooks")
print("Document count in test_index:", response['count'])

Document count in test_index: 1795


In [ ]:
from pprint import pp

# Se imprimen los índices de cada documento
response = es.indices.get_mapping(index="textbooks")
pp(response.body)

{'textbooks': {'mappings': {'properties': {'concepts': {'type': 'text',
                                                        'fields': {'keyword': {'type': 'keyword',
                                                                               'ignore_above': 256}}},
                                           'field': {'type': 'text',
                                                     'fields': {'keyword': {'type': 'keyword',
                                                                            'ignore_above': 256}}},
                                           'markdown': {'type': 'nested',
                                                        'properties': {'chapter': {'type': 'text',
                                                                                   'fields': {'keyword': {'type': 'keyword',
                                                                                                          'ignore_above': 256}}},
                                   

In [ ]:
response = es.indices.get_field_mapping(index="textbooks", fields="concepts")
print("Field Mapping for 'text':")
pp(response.body)

Field Mapping for 'text':
{'textbooks': {'mappings': {'concepts': {'full_name': 'concepts',
                                         'mapping': {'concepts': {'type': 'text',
                                                                  'fields': {'keyword': {'type': 'keyword',
                                                                                         'ignore_above': 256}}}}}}}}


In [ ]:
from elasticsearch import Elasticsearch
from pprint import pp

index_name="textbooks"
# Create an Elasticsearch client
es = Elasticsearch(["https://mrkite-elasticsearch.hf.space"])  # Adjust the host as needed

response = es.indices.put_settings(
    index=index_name,
    body={
        "index": {
            "highlight.max_analyzed_offset": 2000000
        }
    }
)
settings = es.indices.get_settings(index=index_name)
print(settings)


{'textbooks': {'settings': {'index': {'highlight': {'max_analyzed_offset': '2000000'}, 'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'textbooks', 'creation_date': '1733600278042', 'number_of_replicas': '1', 'uuid': 'eIlryJMhTSiwqe9CzDXEzQ', 'version': {'created': '8512000'}}}}}


### Highlighting
https://www.elastic.co/guide/en/elasticsearch/reference/current/highlighting.html

Se realizará la búsqueda de cada query en cada subsección de cada texto subido, evitando que analizar todo el texto para alguna coincidencia

In [ ]:
from elasticsearch import Elasticsearch
from pprint import pp
import re
index_name="textbooks"

# Create an Elasticsearch client
es = Elasticsearch(["https://mrkite-elasticsearch.hf.space"])  # Adjust the host as needed

response = es.indices.put_settings(
    index=index_name,
    body={
        "index": {
            "highlight.max_analyzed_offset": 2000000
        }
    }
)

# Define the search query
def search_context(query):
    query = {
        "query": {
            "nested": {
                "path": "markdown",
                "query": {
                    "match": {
                        "markdown.content": {
                            "query": query,  # The search query text
                        }
                    }
                },
                "inner_hits": {  # Return detailed inner hit results
                    "highlight": {
                        "type": "plain",
                        "fields": {
                            "markdown.content": {
                                "fragment_size": 500,  # Smaller fragment size
                                "number_of_fragments": 3  # Limit to 3 fragments
                            }
                        }
                    }
                }
            }
        }
    }

    # Perform the search on the "test_index" index
    response = es.search(index="textbooks", body=query)


    context = ""
    pp(response)
    # Print results with highlighting
    print(len(response["hits"]["hits"]))
    for hit in response["hits"]["hits"][:min(2, len(response["hits"]["hits"]))]:
        print("-"*20)
        print(f"Document ID: {hit['_id']}")
        print(f"Score: {hit['_score']}")
        # context_titles.append((hit["_source"]["topic"],hit["_score"]))
        # pp(hit["inner_hits"])
        for inner_hit in hit["inner_hits"]["markdown"]["hits"]["hits"]:
            # pp(inner_hit["_source"])
            for fragment in inner_hit["highlight"]["markdown.content"]:
                # pp(fragment)
                fragment = re.sub(r"#.*?(?=\n)","", fragment)
                fragment = re.sub(r"[\n\r]", "", fragment)
                fragment = fragment.replace("<em>", "").replace("</em>", "")
                print(fragment)
                context += "\n" + fragment

    print(context)
    return context


In [ ]:
search_context("What is Newton's second law of motion")

ObjectApiResponse({'took': 72, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1737, 'relation': 'eq'}, 'max_score': 12.012395, 'hits': [{'_index': 'textbooks', '_id': '108', '_score': 12.012395, '_ignored': ['outline.keyword'], '_source': {'topic': 'Dynamics and Vibration (13.013J).', 'model': 'gpt-3.5-turbo-instruct', 'concepts': '[]', 'outline': '# Table of Contents\n- [Dynamics and Vibration: Theory and Applications":](#Dynamics-and-Vibration:-Theory-and-Applications":)\n  - [Foreward](#Foreward)\n  - [Chapter: Dynamics and Vibration: Theory and Applications](#Chapter:-Dynamics-and-Vibration:-Theory-and-Applications)\n    - [Introduction](#Introduction)\n    - [Related Context](#Related-Context)\n    - [Last textbook section content:](#Last-textbook-section-content:)\n  - [Chapter: Dynamics and Vibration: Theory and Applications](#Chapter:-Dynamics-and-Vibration:-Theory-and-Applications)\n    - [Introduction](#Int

"\nIn this section, we will discuss the equations of motion for systems containing rigid bodies. These equations are essential for understanding the behavior of physical systems and predicting their motion.Euler's equations of motion are a set of equations that describe the rotational motion of a rigid body. They are derived from Newton's second law of motion and are fundamental in the study of dynamics.To\n understand Euler's equations, we must first define some key terms. The angular velocity of a rigid body is the rate of change of its angular position with respect to time. It is denoted by $\\omega$ and is measured in radians per second. The angular acceleration of a rigid body is the rate of change of its angular velocity with respect to time. It is denoted by $\\alpha$ and is measured in radians per second squared.Now, let's consider a rigid body rotating about a fixed axis. The axis of\n rotation is known as the principal axis, and the moment of inertia of the body about this ax

In [ ]:
qa_pipeline = pipeline("question-answering", model="MrKite/bert-large-squadv2")
question = "What is the first law of thermodynamics?"

# Get the answer
answer = qa_pipeline(question=question, context=context)
print(f"Question: {question}")
print(f"Answer: {answer}")

Ejemplo de búsqueda en cada campo

In [ ]:
from elasticsearch import Elasticsearch
from pprint import pp
index_name="textbooks"

# Create an Elasticsearch client
es = Elasticsearch(["https://mrkite-elasticsearch.hf.space"])  # Adjust the host as needed

response = es.indices.put_settings(
    index=index_name,
    body={
        "index": {
            "highlight.max_analyzed_offset": 2000000
        }
    }
)

# Define the search query
query = {
    "query": {
        "match": {
            "markdown": "Archimedes' principle"
        },

    },
    "highlight": {
        "type": "plain",
        "fields": {
            "markdown": {
                "fragment_size": 500,  # Smaller fragment size
                "number_of_fragments": 3
            }
        }
    },
}

# Perform the search on the "test_index" index
response = es.search(index="textbooks", body=query)

# Print results with highlighting
print(len(response["hits"]["hits"]))
for hit in response["hits"]["hits"][:min(5, len(response["hits"]["hits"]))]:
    print("-"*20)
    print(f"Document ID: {hit['_id']}")
    print(f"Score: {hit['_score']}")
    hit["_source"]["outline"] = ["Indices"]
    hit["_source"]["markdown"] = ["Texto de libro"]
    pp(hit["_source"])

    print("Matched Text Fragments with Highlighting:")
    for fragment in hit["highlight"]["markdown"]:
        print("-"*10)
        print(fragment)
        m = fragment.index("<em>")
        # print(fragment[min(0, m-50):max(len(fragment), m+50)])


10
--------------------
Document ID: 102
Score: 12.145523
{'topic': 'Principles of Naval Architecture.',
 'model': 'gpt-4-0613',
 'concepts': '[]',
 'outline': ['Indices'],
 'markdown': ['Texto de libro'],
 'field': 'mechanical_engineering',
 'subfield': 'fluid_mechanics',
 'rag': 'N/A'}
Matched Text Fragments with Highlighting:
----------
 <em>Archimedes</em>' <em>principle</em> and the equation of hydrostatic equilibrium, will be presented and explained. For instance, <em>Archimedes</em>' <em>principle</em>, which can be expressed as `$\Delta w = \rho_f \cdot g \cdot V$`, where `$\Delta w$` is the weight of the displaced fluid, `$\rho_f$` is the density of the fluid, `$g$` is the acceleration due to gravity, and `$V$` is the volume of the displaced fluid.



By the end of this chapter, readers should have a solid understanding of the principles of hydrostatics and their
----------
$` is the height of the fluid column above the point in question. This equation tells us that the pressu